### Collisional-radiative model for Argon Plasma
#### Steady 1D shock: FOM vs. ROM

In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
style = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle"
plt.style.use(style)

In [3]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import utils
from romar import systems
from romar import postproc as pp

Set enviroment

In [4]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

Set inputs

In [5]:
paths = {
  # > Solutions
  "sols": {
    "FOM": "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/cr_argon/testcases/1d_shock_steady/output_shock_fom/",
    "CoBRAS": "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/cr_argon/testcases/1d_shock_steady/output_shock_rom/"
  },
  # > Thermochemical database
  "dtb": "/home/zanardi/Codes/ML/ROMAr/romar/examples/database",
  # > Output folder
  "out": "./1d_shock_steady_figs/"
}
# Names of FOM and ROM
names = {
  "FOM": "FOM",
  "ROM": "CoBRAS"
}
# Space limits [m]
xlim = [1e-6, 3e-1]
# Names of variables
labels = {
  "Th": "T_h",
  "Te": "T_e",
  "Ar": "\\text{Ar}",
  "Arp": "\\text{Ar$^+$}",
  "em": "\\text{e$^-$}"
}
# Reference values for error
hline = {
  "temp": 1.0,
  "mom": 1.0,
  "dist": 10.0
}
# Epsilon value for error calculation
eps = 1e-8

In [6]:
os.makedirs(paths["out"], exist_ok=True)

Initialize 0D thermochemical system

In [7]:
system = systems.BoxAd(
  species={k: paths["dtb"] + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=paths["dtb"] + "/rates/kin_fit.p",
  rad_dtb=paths["dtb"] + "/rates/rad_fit.p",
  use_rad=True,
  use_tables=False
)

Read FOM and ROM solutions

In [8]:
def read_sol(path, system, xlim=None):
  # Load shock solution
  columns = "x X_em X_Ar X_Arp u Th Te rho p n H Mf".split()
  sol = np.loadtxt(path + "/shock.dat")
  sol = pd.DataFrame(data=sol, columns=columns)
  # Read locations, velocity, and temperatures
  dsol = {k: sol[k].values.reshape(-1) for k in ("x", "u", "Th", "Te")}
  # Read populations
  n = {k: read_pop(path, k).T for k in ("Ar", "Arp")}
  n["em"] = (sol["X_em"] * sol["n"]).values
  dsol["n"] = np.vstack([n[k] for k in system.species_order]).T
  # Space window
  if (xlim is not None):
    i = (dsol["x"] >= np.amin(xlim)) * (dsol["x"] <= np.amax(xlim))
    dsol = {k: x[i] for k, x in dsol.items()}
  # Return
  dsol["n"] = dsol["n"].T
  return dsol

def read_pop(path, species):
  ni_gi  = np.loadtxt(path + f"/pop_{species}.dat", comments="&", skiprows=2)[:,1]
  levels = system.mix.species[species].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [9]:
sols = {k: read_sol(path, system, xlim) for (k, path) in paths["sols"].items()}
x = sols[names["FOM"]]["x"]

Postprocess solutions

In [10]:
# Primitive variables
prim = {k: [sol[i] for i in ("n", "Th", "Te")] for (k, sol) in sols.items()}
# Postprocessed solution
data = {k: system.postproc_sol(*p) for (k, p) in prim.items()}
# Error between ROM and FOM
err = {
  names["ROM"]: {
    "mom": system.compute_err_mom(
      n_true=prim[names["FOM"]][0],
      n_pred=prim[names["ROM"]][0],
      eps=eps
    ),
    "dist": system.compute_err_mom(
      n_true=prim[names["FOM"]][0],
      n_pred=prim[names["ROM"]][0],
      eps=eps
    ),
    "temp": system.compute_err_temp(
      Ti_true=prim[names["FOM"]][1:],
      Ti_pred=prim[names["ROM"]][1:],
      eps=eps
    )
  }
}

Plot solutions

> Temperatures

In [11]:
pp.plot_temp_evolution(
  path=paths["out"],
  x=x,
  y=data,
  err=err,
  xlabel=r"$x$ [m]",
  xlim=[xlim[0],3e-2],
  # ylim_err=[1e-7,3e0],
  xscale="linear",
  err_scale="log",
  legend_loc="lower right",
  hline=hline,
)

> Velocity

In [12]:
u_path = paths["out"] + "/vel/"
os.makedirs(u_path, exist_ok=True)
u_sol = {k: sol["u"] for (k, sol) in sols.items()}
u_err = {names["ROM"]: utils.absolute_percentage_error(
  y_true=sols[names["FOM"]]["u"],
  y_pred=sols[names["ROM"]]["u"],
  eps=eps
)}
pp.plot_evolution(
  x=x,
  y=u_sol,
  xlim=xlim,
  labels=[r"$x$ [m]", "$u$ [m/s]"],
  legend_loc="best",
  scales=["linear", "linear"],
  figname=u_path + "/sol",
  save=True,
  show=False
)
pp.plot_evolution(
  x=x,
  y=u_err,
  xlim=xlim,
  ylim=None,
  hline=1.0,
  labels=[r"$x$ [m]", "$u$ error [\%]"],
  legend_loc="lower right",
  scales=["linear", "log"],
  figname=u_path + "/err",
  save=True,
  show=False
)

> Moments

In [13]:
pp.plot_mom_evolution(
  path=paths["out"],
  x=x,
  y=data,
  err=err,
  species=system.mix.species,
  labels=labels,
  xlabel=r"$x$ [m]",
  ylim_err=None,
  err_scale="log",
  hline=hline,
  max_mom=2
)